In [16]:
%pip install -q langchain-elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from uuid import uuid4
from elasticsearch import Elasticsearch
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import ElasticSearchBM25Retriever

**Загрузка базы знаний:**

In [28]:
with open(
    file=r"C:\Users\andre\AdvancedRAG\static\texts\ТИУ База знаний.txt",
    mode="r",
    encoding="utf-8"
) as file:
    text = file.read()

print(f"Длина текста: {len(text)}")

Длина текста: 71987


**Разделение текста на чанки:**

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

chunks = text_splitter.create_documents([text])

N = 5

print(f"Всего чанков: {len(chunks)}")
print(f"Первые {N} чанков:")
chunks[:N]

Всего чанков: 110
Первые 5 чанков:


[Document(page_content='Основные сведения:\nПолное наименование образовательной организации\nФедеральное государственное бюджетное образовательное учреждение высшего образования «Тюменский индустриальный университет»\nНаименование организации на иностранном языке\nFederal State Budget Educational Institution of Higher Education «Industrial University of Tyumen»/ IUT\nСокращенное наименование образовательной организации (при наличии)\nФГБОУ ВО «Тюменский индустриальный университет», ФГБОУ ВО «ТИУ», Тюменский индустриальный университет, ТИУ\nДата создания образовательной организации: 08 января 1964 г.\nКраткая информация об образовательной организации:\nТюменский индустриальный университет – опорный вуз региона, лидер в подготовке инженерных кадров.\nИнформация о месте нахождения образовательной организации:\nЮридический адрес: 625000, Уральский федеральный округ, Тюменская область, г. Тюмень, ул. Володарского, 38\nФактический адрес: 625000, Уральский федеральный округ, Тюменская область

**Загрузка Embedding мoдели:**

In [30]:
embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={'normalize_embeddings': False},
)

**Подключение в базе данных (Elastic-search):**

In [31]:
elastic_client = Elasticsearch(
    hosts="http://localhost:9200",
    basic_auth=("elastic", "password")
)


elastic_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="tyuiu_index",
    embedding=embeddings,
    es_user="elastic",
    es_password="password",
)

**Загрузка документов:**

In [32]:
import os


def read_files_in_directory(directory_path: str) -> str:
    files = []
    if not os.path.exists(directory_path):
        return
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        files.append(file_path)
    return files


directory_path = r'C:\Users\andre\AdvancedRAG\static\chanks'
files = read_files_in_directory(directory_path)

In [33]:
def get_file_name(file_path: str) -> str:
    return file_path.split("\\")[-1]

In [35]:
elastic_client.indices.delete(index="tyuiu_index")

ObjectApiResponse({'acknowledged': True})

In [36]:
elastic_store.add_documents(documents=chunks)

['210a01dd-572d-487c-9c36-5b789fdcd377',
 '0432c131-a2c7-4a33-99b0-8a3dae9d4f3b',
 '0f7e9950-f068-4ffc-8d7f-266c9e79092c',
 '74689928-d306-43a2-9cbd-64776e1f73eb',
 '7b19c8c6-c6b1-43fb-9bf5-00a40d979cb8',
 'aede2429-0f9d-41c1-adab-de3ac73f3249',
 '6198d350-4fe2-49be-ba42-592258236925',
 '6da74d0f-2c8c-4cd0-b716-8123b5a8f796',
 'eef8098e-14cb-4390-877c-3fefec14a9b1',
 'fef18f4e-4a6d-499f-a5fa-041e712431bd',
 '227714fe-cbfb-4cd6-b0bb-5874e47716ae',
 '0d348340-663d-4270-a35f-7f73c4352d4a',
 '0a31a9b8-33de-4588-8e8a-3ec3133ae0c0',
 '3a2c1433-05f6-4692-8e12-d3986a6fe772',
 'c22baee4-86e9-4516-8eb2-16bfb3275d82',
 '96688e2f-0202-4662-a738-44b3b6b0b3f5',
 'b9605b98-dd0f-4ae6-b343-fe3dafddd054',
 '82b6568c-46e2-4512-b836-77642c5817bd',
 '5a1b3a5e-44a9-4649-b5c9-3cd4351b1e16',
 '2261fbca-7aee-4bf5-a66c-044cdf6af47b',
 '73ca3a9c-641b-42fc-9250-c82bbbc7b351',
 'd8d2dc95-905d-47c0-af18-f6ff475250d1',
 'd8537f64-97ca-458c-b93a-9597f02dd27b',
 '8d9b913d-1976-4564-ac47-b40a50efbd47',
 '08016ab8-6b30-

In [37]:
docs = elastic_store.similarity_search("какой минимальный проходной балл на направление архитектура")
docs

[Document(page_content='Преимущества обучения в Центре архитектурной подготовки:\n- высококвалифицированный педагогический состав, в том числе члены Союза архитекторов, дизайнеров и художников России;\n- программа обучения базируется на специфике архитектурного и дизайнерского творчества;\n- благодаря полученным навыкам выпускники центра поступают в профильные ВУЗы страны и начинают свою деятельность в области архитектуры и дизайна;\n- в 2024 году 95% абитуриентов, прошедших подготовительные курсы ЦАП АРХИД ТИУ, успешно сдали творческие вступительные испытания'),
 Document(page_content='Творческие направления, такие как «Дизайн интерьера» и «Дизайн городской среды», имеют проходной балл 51 (платный) и 75 (бюджетный).\n\nОбратите внимание, что для некоторых направлений указаны баллы только для одного вида обучения (например, «Моделирование механических систем и процессов» — только бюджетное, «Машины и оборудование нефтяных и газовых промыслов» — только платное), либо баллы не указаны во

In [38]:
elastic_client.indices.delete(index="docs-tyuiu-index")

ObjectApiResponse({'acknowledged': True})

In [39]:
bm25_retriever = ElasticSearchBM25Retriever(
    client=elastic_client, 
    index_name="docs-tyuiu-index",
    search_kwargs={"k": 2}
)

In [40]:
bm25_retriever.add_texts([document.page_content for document in chunks])

['25467328-29fb-4818-b939-e6af5ca5674e',
 '3a5d970a-cef2-42f6-b4b0-ee43eb50d563',
 '6c421e90-e20b-41f4-966e-7bc45f8d94c3',
 '9b9bc43c-1cdd-4f9a-a03e-c92eca352e9f',
 'fe113a03-38fa-4ca9-8db4-ddaf6c71d00b',
 '4ff3f64d-0c69-447f-9bc1-fd32f81346e3',
 'edea6c08-b7ec-478f-bbbd-7edff0bdbce7',
 '6405a294-03fc-447f-833f-9437f11e041d',
 '91cfb603-febb-4806-9cab-2ddfdadee1a6',
 'f39e8146-06ff-4bee-88f1-21544604ec67',
 'c41f987f-b9f7-4128-a1ac-a7a4b8720bde',
 '180a22ca-2f18-4e35-bea0-1d12b641e788',
 '293bd2e0-2171-4db2-82f3-10bc61f796ce',
 '471b15f9-2447-4283-8483-5c4833ef40dc',
 'abbd0f34-13fb-406c-a6be-0654152d0f81',
 'f7b4d89d-120b-478c-9ac0-186064a9b022',
 '193745f4-d852-4e06-9504-1a3b59754ccd',
 'b040f592-7c85-458e-9f3a-814e957eb6fb',
 '4b3ddeb9-26f8-4b24-a3db-6002db69323f',
 'f1d84448-f0a4-49cd-9d90-1373c11b988f',
 'dad4eb2b-f288-4bba-9cbf-8df80a282f8d',
 '7fa9176b-69e9-4cec-bbb1-0755f6a81d44',
 'afbcbbb9-8863-40ca-b64b-35ee9f186e65',
 '6527ca37-c05e-4fab-ad38-59cc43b6d096',
 '95ee9378-8d93-

In [41]:
vector_store_retriever = elastic_store.as_retriever(search_kwargs={"k": 2})

In [42]:
from langchain.retrievers import EnsembleRetriever

In [43]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[vector_store_retriever, bm25_retriever],
    weights=[0.6, 0.4]
)

In [49]:
docs = ensemble_retriever.invoke("Какой минимальный балл на архитектуру")
for doc in docs:
    print(doc)

page_content='Минимальный балл ЗФО:'
page_content='Преимущества обучения в Центре архитектурной подготовки:
- высококвалифицированный педагогический состав, в том числе члены Союза архитекторов, дизайнеров и художников России;
- программа обучения базируется на специфике архитектурного и дизайнерского творчества;
- благодаря полученным навыкам выпускники центра поступают в профильные ВУЗы страны и начинают свою деятельность в области архитектуры и дизайна;
- в 2024 году 95% абитуриентов, прошедших подготовительные курсы ЦАП АРХИД ТИУ, успешно сдали творческие вступительные испытания'
page_content='Для поступления на направление “Архитектура” (07.03.01) нужно сдать два дополнительных вступительных испытания творческой направленности: “Композицию” и “Рисунок”. Минимальный балл для каждого из них - 40, максимальный - 100. Оба испытания проводятся в форме письменной работы на русском языке.
Для поступления на направление “Дизайн архитектурной среды” (07.03.03) также необходимо сдать два до